<a id="1"></a>
<div style="padding:20px;
            color:#F0F3FF;
            margin:10;
            font-size:200%;
            text-align:left;
            display:fill;
            border-radius:15px;
            background-color:#211951;
            overflow:hidden;
            font-weight:900">
            <span style='color:#15F5BA'></span>London Crime Analysis
</div>
<br>Author : <i>Bachtiyar M. Arief</i>

<a id="A"></a>
# <b>A. <span style='color:#E1B12D'>Latar Belakang</span></b> 

Kejahatan di area metropolitan, seperti <b>London</b>, terjadi dalam pola yang berbeda. Terdapat sebuah data yang mencakup jumlah laporan kejahatan menurut bulan, wilayah LSOA, dan kategori besar/kecil dari Januari 2008 - Desember 2016. Sasaran utama analisis ini adalah untuk memberikan wawasan mendalam tentang pola kejahatan di berbagai area di London dari tahun 2008 hingga 2016. Target pengguna analisis ini adalah pejabat polisi dan pembuat kebijakan

<a id="B"></a>
# <b>B. <span style='color:#F6F193'>Setup Awal</span></b> 

In [73]:
import os
import warnings
import plotly.offline as pyo

current_path = os.getcwd()
os.chdir(current_path)

warnings.filterwarnings('ignore')
pyo.init_notebook_mode(connected=True)

<a id="C"></a>
# <b>C. <span style='color:#E1B12D'>Ekstraksi Data</span></b> 

Data yang diekstrak merupakan data yang diambil dari https://www.kaggle.com/datasets/jboysen/london-crime/data. Data dengan format .csv ini memuat 13 juta baris berisi jumlah laporan kriminal berdasarkan bulan, wilayah LSOA, dan kategori utama/kecil

In [74]:
# Import library yang dibutuhkan
import pandas as pd

# Proses ekstraksi data
data_crime = pd.read_csv(
    filepath_or_buffer = 'london_crime.csv',
    dtype = {
        'year' : object,
        'month' : object
    }
)

# Urutkan data berdasarkan tahun dan bulan
data_crime = data_crime.sort_values(
    by = ['year', 'month', 'lsoa_code'],
    ignore_index = True
)

# Tampilkan sampel data
data_crime.head()

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01000001,City of London,Violence Against the Person,Wounding/GBH,0,2008,1
1,E01000001,City of London,Drugs,Drug Trafficking,0,2008,1
2,E01000001,City of London,Theft and Handling,Theft/Taking Of Motor Vehicle,0,2008,1
3,E01000001,City of London,Criminal Damage,Criminal Damage To Other Building,0,2008,1
4,E01000001,City of London,Other Notifiable Offences,Other Notifiable,0,2008,1


<a id="D"></a>
# <b>D. <span style='color:#E1B12D'>Informasi Umum Data</span></b> 

<a id="D.1."></a>
## <b>D.1. <span style='color:#F6F193'><i>Data Dictionary</i></span></b> 

* `lsoa_code` : Kode LSOA (Lower Layer Super Output Area) adalah kode unik yang mengidentifikasi area geografis kecil di Inggris.
* `borough` : Nama borough atau wilayah administratif di London
* `major_category` : Kategori utama dari jenis kejahatan
* `minor_category` : Kategori lebih spesifik dari jenis kejahatan dalam kategori utama
* `value` : Jumlah insiden kejahatan untuk kategori tertentu di area dan waktu tertentu
* `year` : Tahun di mana kejahatan terjadi
* `month` : Bulan di mana kejahatan terjadi

<a id="D.2."></a>
## <b>D.2. <span style='color:#F6F193'>Tipe Data</span></b> 

In [75]:
# Informasi umum pada data
data_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13490604 entries, 0 to 13490603
Data columns (total 7 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   lsoa_code       object
 1   borough         object
 2   major_category  object
 3   minor_category  object
 4   value           int64 
 5   year            object
 6   month           object
dtypes: int64(1), object(6)
memory usage: 720.5+ MB


<a id="D.3."></a>
## <b>D.3. <span style='color:#F6F193'>Isi Data</span></b> 

In [37]:
# Seleksi hanya kolom bertipe non-numerik
obj_columns = data_crime.select_dtypes(exclude = ['number']).columns

for obj in obj_columns:
    if (obj not in ['lsoa_code']):
        nilai_unik = data_crime[obj].astype("str").unique()
        print(f'Kolom \033[33;1m{obj}\033[0m')
        print(f'Terdapat {len(nilai_unik)} nilai unik')
        print(f'Isi : \033[3m{", ".join(nilai_unik)}\033[0m\n')

Kolom borough
Terdapat 33 nilai unik
Isi : City of London, Barking and Dagenham, Barnet, Bexley, Brent, Bromley, Camden, Croydon, Ealing, Enfield, Greenwich, Hackney, Hammersmith and Fulham, Haringey, Harrow, Havering, Hillingdon, Hounslow, Islington, Kensington and Chelsea, Kingston upon Thames, Lambeth, Lewisham, Merton, Newham, Redbridge, Richmond upon Thames, Southwark, Sutton, Tower Hamlets, Waltham Forest, Wandsworth, Westminster

Kolom major_category
Terdapat 9 nilai unik
Isi : Violence Against the Person, Drugs, Theft and Handling, Criminal Damage, Other Notifiable Offences, Burglary, Robbery, Sexual Offences, Fraud or Forgery

Kolom minor_category
Terdapat 32 nilai unik
Isi : Wounding/GBH, Drug Trafficking, Theft/Taking Of Motor Vehicle, Criminal Damage To Other Building, Other Notifiable, Common Assault, Burglary in Other Buildings, Other Theft, Other Criminal Damage, Theft/Taking of Pedal Cycle, Harassment, Theft From Shops, Personal Property, Other Theft Person, Other viole

Tindak Kriminal yang ada pada data (<i>Major Category</i>) diantaranya adalah :

* <span style='color:#ed4d4d'><b><i>Burglary</b></i></span> (Pencurian dengan Paksaan)
* <span style='color:#ed4d4d'><b><i>Criminal Damage</b></i></span> (Kerusakan Kriminal)
* <span style='color:#ed4d4d'><b><i>Drugs</b></i></span> (Narkoba)
* <span style='color:#ed4d4d'><b><i>Fraud or Forgery</b></i></span> (Penipuan atau Pemalsuan)
* <span style='color:#ed4d4d'><b><i>Other Notifiable Offences</b></i></span> (Tindak Kriminal Lain yang Harus Dilaporkan)
* <span style='color:#ed4d4d'><b><i>Robbery</b></i></span> (Perampokan)
* <span style='color:#ed4d4d'><b><i>Sexual Offences</b></i></span> (Tindak Kriminal Seksual)
* <span style='color:#ed4d4d'><b><i>Theft and Handling</b></i></span> (Pencurian dan Penanganan Barang Curian)
* <span style='color:#ed4d4d'><b><i>Violence Against the Person</b></i></span> (Kekerasan Terhadap Orang)

<a id="D.4."></a>
## <b>D.4. <span style='color:#F6F193'>Periksa <i>Missing Value</i> & Duplikasi Data</span></b> 

In [40]:
# Periksa missing value pada tiap kolom
for column in data_crime.columns:
    total_null = data_crime[column].isna().sum()
    print(f'Kolom \033[33;1m{column}\033[0m memiliki {total_null} \033[3mmissing value\033[0m')

Kolom lsoa_code memiliki 0 missing value
Kolom borough memiliki 0 missing value
Kolom major_category memiliki 0 missing value
Kolom minor_category memiliki 0 missing value
Kolom value memiliki 0 missing value
Kolom year memiliki 0 missing value
Kolom month memiliki 0 missing value


In [42]:
# Periksa duplikasi data
print(f'Terdapat {data_crime.duplicated().sum()} data yang duplikat')

Terdapat 0 data yang duplikat


<a id="E"></a>
# <b>E. <span style='color:#E1B12D'><i>Feature Engineering</i></span></b> 

<a id="E.1."></a>
## <b>E.1. <span style='color:#F6F193'>Tambahkan Kolom Periode</span></b> 

Kolom periode adalah kolom gabungan antara tahun dan bulan dengan format %Y-%m

In [80]:
# Tambahkan kolom period (%Y-%m)
data_crime['period'] = pd.to_datetime(data_crime['year'] + '-' + data_crime['month'], format = '%Y-%m')
data_crime['period'] = data_crime['period'].dt.strftime('%Y-%m')

<a id="E.2."></a>
## <b>E.2. <span style='color:#F6F193'>Tambahkan Kolom Nama Bulan</span></b> 

Tambahkan kolom nama bulan, dengan melakukan pemetaan pengkodean 1 untuk Januari, 2 untuk Februari, ... dst


In [81]:
# Tambahkan kolom nama bukan
map_month_name = {    
    '1' : 'January',
    '2' : 'February',
    '3' : 'March',
    '4' : 'April',
    '5' : 'May',
    '6' : 'June',
    '7' : 'July',
    '8' : 'August',
    '9' : 'September',
    '10' : 'October',
    '11' : 'November',
    '12' : 'December'
}

data_crime['month_name'] = data_crime['month'].map(map_month_name)
data_crime['month_name'] = pd.Categorical(
    data_crime['month_name'],
    categories = map_month_name.values(),
    ordered = True
)

<a id="E.3."></a>
## <b>E.3. <span style='color:#F6F193'>Ekstraksi Data Geografis</span></b> 

Data geografis digunakan untuk membuat grafik peta (*cloropeth map*)


In [82]:
import json 

# Ekstraksi data geojson
with open('London.geojson') as f:
    london_geo = json.load(f)

<a id="F"></a>
# <b>F. <span style='color:#E1B12D'><i>Exploratory Data Analysis</i></span></b> 

<a id="F.1."></a>
## <b>F.1. <span style='color:#F6F193'><i>Trend</i> Kejadian Kriminal Tiap Tahun</span></b> 

Analisis tren memungkinkan untuk mengidentifikasi berapa banyak tindak kriminal yang terjadim jenis kejahatan yang paling menonjol, lokasi rawan, serta pola-pola baru yang muncul. Dengan menganalisis tren kejadian kriminal secara komprehensif, pemerintah, kepolisian, dan masyarakat dapat bekerja sama untuk mengembangkan strategi yang lebih efektif dalam menjaga keamanan dan ketertiban di London & membantu mengetahui tantangan-tantangan keamanan yang dihadapi sehingga dapat ditangani dengan lebih efektif.

In [76]:
# Hitung total kejadian kriminal tiap tahun
total_crime_per_year = data_crime.groupby(['year'], as_index = False).agg(
    total_crime = ('value', 'sum')
)

total_crime_per_year['presentase'] = total_crime_per_year['total_crime'].pct_change().round(4) * 100
total_crime_per_year['presentase'] = total_crime_per_year['presentase'].map(lambda x: f'{x:.2f}%')
total_crime_per_year['color'] = total_crime_per_year['total_crime'].apply(lambda x: '#BA0001' if x == total_crime_per_year['total_crime'].max() else '#F7B4BB')

In [130]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = px.bar(
    total_crime_per_year,
    x = 'year',
    y = 'total_crime',
    orientation = 'v',
    text_auto = True,
    hover_data = ['presentase']
)

fig.update_layout(
    width = 950,
    height = 500,
    xaxis_title = '',
    yaxis_title = '',
    showlegend = False,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = '<b><span style="color:#BA0001">Total Kejadian Kriminalitas</span> Tiap Tahun</b><br><sup><sup>Periode 2008 - 2016</sup></sup>',
        font = dict(
            size = 25,
            color = '#B0A7A7'
        ),
        y = 0.95,
        x = 0.075
    )
)

fig.update_yaxes(
    showticklabels = False,
    range = [620000, 740000], 
    automargin = True
)

fig.update_traces(
    marker_color = total_crime_per_year['color'].tolist(),
    texttemplate='%{y:.0f}',
    textposition = 'inside',
    hovertemplate = '<b>%{label}</b><br>Total Crime = %{value:.0f}<br>Growth YtD = %{customdata[0]}'
)

fig.show()

***Analisa :***<br><br>
Pada periode 2008 hingga 2016, tercatat bahwa tahun 2008 mencatatkan angka kriminial tertinggi yakni sebesar 738K kasus kriminal terjadi. Jika diamati ada pola yang cukup menarik yakni terdapat semacam siklus 4 tahunan dan 'kebetulan' angka kriminal terjadi cukup tinggi di tahun kabisat (2008, 2012 dan 2016) lalu *trend* menunjukan penurunan setelahnya. 

Pada tahun 2008, Inggris mengalami krisis yang disebut **Krisis Subprime Mortgage** yakni krisis keuangan yang terjadi pada 2007-2008 dan merupakan salah satu penyebab utama dari Krisis Keuangan Global 2008. Krisis ini berakar dari kegagalan dalam pasar perumahan di Amerika Serikat dan memiliki dampak luas di seluruh sistem keuangan global. Krisis tersebut memungkinkan jika menjadi salah satu penyebab angka kriminalitas pada tahun 2008 sangat tinggi.

In [128]:
# Import library plotly untuk visualisasi data
import numpy as np
import plotly.express as px

def plot_timeseries(
    data : pd.DataFrame,
    value : str,
    judul : str,
    warna : str = 'black',
    showtrend : bool = False,
):
    fig = px.line(
        data_frame = data,
        x = data.index,
        y = value,
        color_discrete_sequence = [warna]
    )

    if(showtrend):
        # Hitung koefisien regresi linier
        coefficients = np.polyfit(range(len(data)), data[value], 1)
        slope = coefficients[0]
        intercept = coefficients[1]

        # Tambahkan garis regresi ke plot
        fig.add_scatter(
            x = data.index,
            y = range(len(data)) * slope + intercept,
            mode = 'lines',
            line = dict(
                color = '#F7B4BB',
                dash = 'dash'
            )
        )

    fig.update_layout(
        width = 950,
        height = 450,
        showlegend = False,
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        title = dict(
            text = judul,
            font = dict(
                size = 28,
                color = '#B0A7A7'
            ),
            y = 0.92,
            x = 0.5
        ),
        yaxis = dict(
            title = '',
            showgrid = False,
            showline = False,
            showticklabels = False,
            zeroline = False,
        ),
        margin = dict(
            t = 80,
            b = 10,
            r = 20
        )
    )

    fig.show()

In [129]:
# Agregasi (Total Crime per Month-Year)
total_crime_over_times = data_crime.groupby(['year', 'period'], as_index = False).agg(
    total_crime = ('value', 'sum')
)

# Plot Grafik Time Series
plot_timeseries(
    data = total_crime_over_times.set_index('period'),
    value = 'total_crime',
    judul = f"<b><span style='color:#BA0001'>Total Kriminalitas</span> di London</b><br><sup><sup>2008 - 2016</sup></sup>",
    warna = '#BA0001',
    showtrend = True
)

***Analisa :***<br><br>
Meskipun diperoleh garis trend yang menurun, namun pada kenyataannya kejadian kriminal tidak signifikan turun terus menerus (masih fluktuatif).

<a id="F.2."></a>
## <b>F.2. <span style='color:#F6F193'>Bagaimana Pola Tindak Kriminal di London Tiap Bulannya selama tahun 2008-2016?</span></b> 

In [127]:
total_crime_over_times = data_crime.groupby(['year', 'month_name'], as_index = False).agg(
    total_crime = ('value', 'sum')
)

total_crime_over_times['presentase'] = total_crime_over_times['total_crime'].pct_change().round(4) * 100
total_crime_over_times['presentase'] = total_crime_over_times['presentase'].map(lambda x: f'{x:.2f}%')

fig = px.line(
    data_frame = total_crime_over_times,
    x = 'month_name',
    y = 'total_crime',
    color = 'year',
    color_discrete_sequence = px.colors.sequential.Reds,
    hover_data = ['presentase', 'year']
)

fig.add_shape(
    type = 'rect',
    x0 = 'February',
    y0 = total_crime_over_times['total_crime'].min() - 1500,
    x1 = 'March',
    y1 = total_crime_over_times['total_crime'].max() + 1500,
    line_width = 0,
    label = dict(
        text = 'Peningkatan Angka<br>Kriminalitas',
        textposition = 'top center',
        font = dict(
            size = 7,
            color = 'red'
        )
    ),
    fillcolor = '#BACDDB',
    opacity = 0.30,
)

fig.add_shape(
    type = 'rect',
    x0 = 'November',
    y0 = total_crime_over_times['total_crime'].min() - 1500,
    x1 = 'December',
    y1 = total_crime_over_times['total_crime'].max() + 1500,
    line_width = 0,
    label = dict(
        text = 'Penurunan Angka<br>Kriminalitas',
        textposition = 'top center',
        font = dict(
            size = 7,
            color = 'green'
        )
    ),
    fillcolor = '#9FE6A0',
    opacity = 0.30,
)

fig.update_layout(
    width = 950,
    height = 450,
    showlegend = True,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = f"<b><span style='color:#BA0001'>Total Kriminalitas</span> per Bulan di London</b><br><sup><sup>2008 - 2016</sup></sup>",
        font = dict(
            size = 28,
            color = '#757882'
        ),
        y = 0.92,
        x = 0.5
    ),
    xaxis = dict(
        title = ''
    ),
    yaxis = dict(
        title = '',
        showgrid = False,
        showline = False,
        showticklabels = False,
        zeroline = False,
    ),
    margin = dict(
        t = 80,
        b = 10,
        r = 20
    )
)

fig.update_traces(
    hovertemplate = '<b>%{x} %{customdata[1]}</b><br>Total Crime = %{y:.0f}<br>Growth MtD = %{customdata[0]}'
)


fig.show()

***Analisa :***<br><br>
Jika di breakdown tiap bulan, ternyata pada bulan Maret terjadi lonjakan kasus kriminal bahkan hampir diseluruh tahun memiliki kemiripan pola. Berbeda dengan saat bulan Desember justru angka kriminalitas menunjukan trend penurunan. Bagi para penegak hukum seperti polisi misalnya, perlu mempersiapkan rencana tanggap darurat untuk merespons secara cepat terhadap lonjakan kejahatan yang mungkin terjadi di bulan Maret. Ini bisa termasuk pengerahan tim khusus atau peningkatan kapasitas operasional.

<a id="F.3."></a>
## <b>F.3. <span style='color:#F6F193'>Selama tahun 2008-2016, Tindak Kriminalitas Apa yang Paling Sering Terjadi?</span></b> 

In [92]:
total_crime_per_major = data_crime.groupby(['major_category', 'minor_category'], as_index = False).agg(
    total_crime = ('value', 'sum')
)

total_crime_per_major = total_crime_per_major.sort_values(by = 'total_crime', ascending = True, ignore_index = True)
total_crime_per_major['minor_category'] = '<b>' + total_crime_per_major['major_category'] + '</b> | ' +  total_crime_per_major['minor_category']
total_crime_per_major = total_crime_per_major.tail(10)

In [123]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = px.bar(
    total_crime_per_major,
    x = 'total_crime',
    y = 'minor_category',
    orientation = 'h',
    text_auto = True
)

fig.update_layout(
    width = 950,
    height = 500,
    xaxis_title = '',
    yaxis_title = '',
    showlegend = False,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = '<b><span style="color:#BA0001">Total Kejadian Kriminalitas</span> per Kategori</b><br><sup><sup>Periode 2008 - 2016</sup></sup>',
        font = dict(
            size = 25,
            color = '#B0A7A7'
        ),
        y = 0.95,
        x = 0.03
    ),
    margin = dict(
        t = 65,
        b = 10
    )
)

fig.update_xaxes(
    showticklabels = False
)

fig.update_traces(
    marker_color = ['#F7B4BB'] * 7 + ['#BA0001'] * 3,
    texttemplate='%{x:.0f}',
    textposition = 'inside',
    hovertemplate = '<b>%{label}</b><br>Total Crime = %{value:.0f}'
)

fig.show()

***Analisa :***<br><br>
**Theft & Handling** (pencurian) menjadi jenis tindak kejahatan yang paling sering terjadi selama 2008 s.d. 2016. Motif pencurian pun beragam seperti pencurian kendaraan bermotor, pencurian toko ataupun pencurian barang lainnya. Disusul oleh **Violence Againts the Person** (kekerasan terhadap orang lain) juga cukup sering terjadi. 

* Penguatan Patroli dan Pengamanan Strategis di daerah-daerah rawan pencurian
* Peningkatan Upaya Pencegahan Kejahatan
* Penguatan Kerja Sama Multistakeholder
* Peningkatan Kapasitas Penegakan Hukum
* Intervensi Rehabilitatif dan Reintegrasi
* Pemantauan dan Evaluasi Berkelanjutan

Dengan menerapkan pendekatan yang komprehensif dan melibatkan berbagai pemangku kepentingan, London dapat secara efektif menangani masalah pencurian dan kekerasan yang menjadi tren kriminal dominan di wilayah tersebut.

<a id="F.4."></a>
## <b>F.4. <span style='color:#F6F193'>Daerah Mana yang Sering Terjadi Tindak Kriminal?</span></b> 

In [94]:
total_crime_per_borough = data_crime.groupby(['borough', 'year'], as_index = False).agg(total_crime = ('value', 'sum'))

total_crime_per_borough = total_crime_per_borough.sort_values(by = 'total_crime', ascending = False, ignore_index = True)
total_crime_per_borough = total_crime_per_borough.rename(columns = {'borough' : 'name'})
total_crime_per_borough

,name,year,total_crime
0,Westminster,2012,59226
1,Westminster,2011,54243
2,Westminster,2013,53852
3,Westminster,2009,48456
4,Westminster,2016,48330
...,...,...,...
292,City of London,2013,110
293,City of London,2011,69
294,City of London,2008,0
295,City of London,2009,0


In [122]:
total_crime_per_borough_over_year = total_crime_per_borough.groupby(['name'], as_index = False).agg(total_crime = ('total_crime', 'sum'))

fig = px.choropleth_mapbox(
    total_crime_per_borough_over_year,
    geojson = london_geo,
    featureidkey = 'properties.name',
    locations = 'name',
    color = 'total_crime',
    color_continuous_scale = 'Reds',
    mapbox_style = 'carto-positron',
    center = {'lat': 51.5074, 'lon': -0.1278},
    zoom = 9,  
    title = 'Total Crime by Region',
)

fig.update_geos(fitbounds='locations', visible=False)

fig.update_layout(
    width = 825,
    height = 800,
    xaxis_title = '',
    yaxis_title = '',
    showlegend = False,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = '<b><span style="color:#BA0001">Total Kejadian Kriminalitas</span> 2008 - 2016</b><br><sup><sup><b>Westminster</b> | Kota Paling Banyak Terjadi Tindak Kriminal</sup></sup>',
        font = dict(
            size = 25,
            color = '#B0A7A7'
        ),
        y = 0.95,
        x = 0.03
    )
)

fig.show()

***Analisa :***<br><br>
Westminster	menjadi daerah yang paling sering terjadi tindak kriminal (455.028 Kejadian) sedangkan daerah yang cukup jarang terjadi adalah City of London (780 Kejadian). City of London didominasi oleh pusat bisnis, perkantoran, dan distrik keuangan, yang cenderung memiliki tingkat pengawasan keamanan yang lebih ketat. Sedangkan Westminister menjadi pusat pemerintahan, pariwisata, dan hiburan, yang cenderung menarik lebih banyak orang dan meningkatkan potensi tindak kejahatan. Secara keseluruhan, perbedaan karakteristik demografis, aktivitas ekonomi, pola aktivitas, dan strategi keamanan antara Westminister dan City of London dapat menjadi faktor-faktor yang berkontribusi pada perbedaan tingkat kriminalitas di kedua wilayah tersebut.

In [96]:
# Memberi rank urutan untuk penjualan best seller pizza
total_crime_per_borough['ranking'] = total_crime_per_borough.groupby(['name'])['total_crime'].transform(sum)\
                                                    .rank(ascending = False, method='dense')\
                                                    .astype(int)
                                                    
total_crime_per_borough = total_crime_per_borough.sort_values(
    by = ['ranking', 'year'],
    ascending = True,
    ignore_index = False
)

In [97]:
end_of_data = total_crime_per_borough[total_crime_per_borough['year'] == '2016']        
end_of_data['avg_total_crime'] = total_crime_per_borough.groupby(['name'])['total_crime'].transform('mean')\
                                                    .round(0).astype(int)                                        

In [120]:
# Definisikan warna garis
pick_color = ['#e81919'] + ['#facccc'] * (end_of_data.shape[0] - 1)

# Buat grafik line dasar
fig = px.line(
    total_crime_per_borough,
    x = 'year',
    y = 'total_crime',
    color = 'name',
    color_discrete_sequence = pick_color,
    hover_data = ['name']
)
# Edit Hover Template
fig.update_traces(
    hovertemplate = '%{customdata[0]}<br>%{x}<br><sup>Total Crime = %{y}</sup>',
    hoverlabel = dict(
        bgcolor = 'white',
        font = dict(
            size = 14,
            color = '#D19C4B'
        )
    )
)

# Definisikan tebal garis
tebal_garis = [8] + [5] * (end_of_data.shape[0] - 1)

# Penebalan Garis yang ingin di highlight
for trace, width in zip(fig.data, tebal_garis):
    if isinstance(trace, go.Scatter):
        trace.line.width = width
        
# Menambahkan titik diujung grafik
fig.add_trace(
    go.Scatter(
        x = end_of_data['year'],
        y = end_of_data['total_crime'],
        mode = 'markers',
        marker = dict(
            size =  [22] + [15] * (end_of_data.shape[0] - 1),
            color = pick_color,
            opacity = 1,
        ),
        text = end_of_data['name'],
        hovertemplate = "%{text}<br>%{x}<br><sup>Total Crime = %{y}</sup>"
    )
)

# Tambahkan annotation
for i in end_of_data['ranking']:
    if(i == end_of_data['ranking'].min()):
        teks = f"<b>{end_of_data[end_of_data['ranking'] == i]['name'].values[0]}</b><br><sup>Avg. Crime = {end_of_data[end_of_data['ranking'] == i]['avg_total_crime'].values[0]}</sup>"
    else:
        teks = f"<b>{end_of_data[end_of_data['ranking'] == i]['name'].values[0]}</b><br>"
    x = end_of_data[end_of_data['ranking'] == i]['year'].values[0]
    y = end_of_data[end_of_data['ranking'] == i]['avg_total_crime'].values[0]
    #"""
    fig.add_annotation(
        x = 8,
        y = end_of_data[end_of_data['ranking'] == i]['total_crime'].values[0],
        text = teks,
        showarrow = False,
        font = dict(
            size = 17,
            color = pick_color[i-1],
            family = "sans serif"
        ),
        align = 'left',
        xshift = 75,
        yshift = 0,
        opacity = 0.8
    )
    #"""
    
fig.update_layout(
    width = 950,
    height = 575,
    showlegend = False,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    yaxis = dict(
        title = 'Total Crime',
        showline = True,
        linewidth = 1,
        color = '#000000',
        linecolor = '#000000'
    ),
    xaxis = dict(
        title = '',
        showline = True,
        linewidth = 1,
        color = '#000000',
        linecolor = '#000000'
    ),
    title = dict(
        text = '<b><span style="color:#BA0001">Total Kejadian Kriminalitas</span> tiap <i>Borough</i> pada 2008 - 2016</b><br><sup><sup><b>Westminster</b> Mendominasi <i>Borough</i> dengan Tingkat Kriminalitas Tertinggi</sup></sup>',
        font = dict(
            size = 25,
            color = '#B0A7A7'
        ),
        y = 0.95,
        x = 0.03
    )
)

fig.show()

***Analisa :***<br><br>
Pada grafik dapat terlihat bahwa tingkat kejahatan di Westminster dari tahun 2008 hingga 2016 selalu menempati posisi pertama. 
Tingginya tingkat kejahatan di Westminster dapat disebabkan oleh kombinasi beberapa faktor termasuk perannya sebagai pusat wisata dan ekonomi utama, lalu lintas pejalan kaki yang tinggi, kehidupan malam yang semarak, serta keberadaan transportasi umum dan acara-acara penting. Elemen-elemen ini berkontribusi pada peluang terjadinya kejahatan dan visibilitas aktivitas kriminal yang lebih tinggi.

Upaya untuk mengelola dan mengurangi kejahatan di Westminster biasanya melibatkan peningkatan kepolisian, keterlibatan masyarakat, dan langkah-langkah untuk mengatasi akar penyebab kejahatan.

<a id="F.5."></a>
## <b>F.5. <span style='color:#F6F193'>Tindak Kriminal Apa yang Sering Terjadi di Westminster?</span></b> 

In [101]:
data_crime_westminster = data_crime[data_crime['borough'] == 'Westminster']
westminster_total_crime = data_crime_westminster.groupby(['major_category'], as_index = False).agg(total_crime = ('value', 'sum'))
westminster_total_crime = westminster_total_crime.sort_values(by = 'total_crime', ascending = False, ignore_index = True)

In [118]:
# Import library untuk visualisasi
import plotly.express as px

# Hitung total data
total_data = westminster_total_crime['total_crime'].sum()

# Warna
hijau_pucat = '#E0ECE4'
merah = '#FF4B5C'

# Buat pie chart
fig = px.pie(
    values = westminster_total_crime['total_crime'],
    names = westminster_total_crime['major_category'],
    color_discrete_sequence = ['#e81919', '#ea3333', '#ed4d4d', '#ef6666', '#f28080', '#f59999', '#f7b3b3', '#facccc', '#fce6e6'],
    hole = 0.65
)

# Atur posisi label
fig.update_traces(
    textposition = 'outside',
    textinfo = 'percent+label',
    hovertemplate='<b>%{label}</b><br>%{value} Kejadian'
)

# Atur luas grafik, hapus legend dan beri judul
fig.update_layout(
    width = 800,
    height = 700,
    showlegend = False,
    margin = dict(l=160, r=200, t=100, b=30),
    title = dict(
        text = f"<b>Proporsi Kategori Tindak Kriminalitas di Westminster, London</b><br><sup><span style='color:{merah}'><b>Theft & Handling</b></span> Menjadi Tindak Kriminal Paling Dominan Terjadi</sup>",
        font = dict(
            size = 20,
            color = '#757882'
        ),
        y = 0.92,
        x = 0.06
    )
)

# Berikan informasi total pelanggan di tengah donut chart
fig.add_annotation(
    text = f'Total Kejadian Kriminal<br><b><span style="font-size: 28px;">{total_data}</b></span>',
    x = 0.5,
    y = 0.5,
    showarrow = False,
    font = dict(size = 20)
)

# Tampilkan grafik
fig.show()

***Analisa :***<br><br>
<b>Theft & Handling</b> menjadi tindak kriminal paling sering terjadi di Westminster, London. Untuk mengatasi tingginya tingkat pencurian dan penanganan barang curian di Westminster, diperlukan pendekatan yang menyeluruh, melibatkan analisis data, peningkatan pengawasan, edukasi masyarakat, kolaborasi komunitas, penegakan hukum yang lebih ketat, dan evaluasi berkelanjutan. Implementasi strategi-strategi ini secara efektif dapat membantu mengurangi kejahatan dan meningkatkan keamanan di area tersebut.

<a id="G"></a>
# <b>G. <span style='color:#E1B12D'><i>Predictive Analytics</i></span></b> 

Akan dilakukan prediksi 2 tahun kedepan tiap jenis kategori tindak kriminal dengan tujuan untuk memperkirakan pola tindak kriminal guna membantu penegak hukum dan lembaga terkait untuk mengambil langkah-langkah preventif untuk mengurangi kejahatan dan meningkatkan keamanan di London.

In [103]:
total_crime_per_majorcat_per_month = data_crime.groupby(['period', 'major_category'], as_index = False).agg(total_crime = ('value', 'sum'))
total_crime_per_majorcat_per_month['type'] = 'original_data'
total_crime_per_majorcat_per_month

,period,major_category,total_crime,type
0,2008-01,Burglary,8396,original_data
1,2008-01,Criminal Damage,8061,original_data
2,2008-01,Drugs,5230,original_data
3,2008-01,Fraud or Forgery,2671,original_data
4,2008-01,Other Notifiable Offences,789,original_data
...,...,...,...,...
967,2016-12,Other Notifiable Offences,1171,original_data
968,2016-12,Robbery,2238,original_data
969,2016-12,Sexual Offences,0,original_data
970,2016-12,Theft and Handling,25728,original_data


In [104]:
from prophet import Prophet

iterasi = 1
data_predict = pd.DataFrame()

for category in total_crime_per_majorcat_per_month['major_category'].unique():
    data_category = total_crime_per_majorcat_per_month[total_crime_per_majorcat_per_month['major_category'] == category]
    data_category = data_category.rename(columns = {
        'period' : 'ds',
        'total_crime' : 'y'
    })
    model = Prophet()
    model.fit(data_category)
    future = model.make_future_dataframe(periods = 24, freq = 'M')
    forecast = model.predict(future)
    forecast['major_category'] = category
    if(category in ['Sexual Offences', 'Fraud or Forgery']):
        forecast['yhat'] = 0
    dump_forecast = forecast[forecast['ds'] >= '2017-01-01'][['ds', 'major_category', 'yhat']]
    if(iterasi == 1):
        data_predict = dump_forecast
    else:
        data_predict = pd.concat([data_predict, dump_forecast])
    iterasi += 1

22:12:08 - cmdstanpy - INFO - Chain [1] start processing
22:12:08 - cmdstanpy - INFO - Chain [1] done processing
22:12:08 - cmdstanpy - INFO - Chain [1] start processing
22:12:08 - cmdstanpy - INFO - Chain [1] done processing
22:12:08 - cmdstanpy - INFO - Chain [1] start processing
22:12:08 - cmdstanpy - INFO - Chain [1] done processing
22:12:08 - cmdstanpy - INFO - Chain [1] start processing
22:12:08 - cmdstanpy - INFO - Chain [1] done processing
22:12:08 - cmdstanpy - INFO - Chain [1] start processing
22:12:08 - cmdstanpy - INFO - Chain [1] done processing
22:12:08 - cmdstanpy - INFO - Chain [1] start processing
22:12:08 - cmdstanpy - INFO - Chain [1] done processing
22:12:08 - cmdstanpy - INFO - Chain [1] start processing
22:12:08 - cmdstanpy - INFO - Chain [1] done processing
22:12:08 - cmdstanpy - INFO - Chain [1] start processing
22:12:08 - cmdstanpy - INFO - Chain [1] done processing
22:12:08 - cmdstanpy - INFO - Chain [1] start processing
22:12:08 - cmdstanpy - INFO - Chain [1]

In [105]:
data_predict['ds'] = pd.to_datetime(data_predict['ds'], format='%Y-%m-%d')
data_predict['ds'] = data_predict['ds'].dt.strftime('%Y-%m')
data_predict['type'] = 'prediction'
data_predict['yhat'] = data_predict['yhat'].round(0).astype(int)
data_predict = data_predict.reset_index(drop = True)
data_predict = data_predict.rename(columns = {
    'ds' : 'period',
    'category' : 'major_category',
    'yhat' : 'total_crime'
})

In [106]:
first_data_predict = total_crime_per_majorcat_per_month[total_crime_per_majorcat_per_month['period'] == '2016-12']
first_data_predict['type'] = 'prediction'
data_predict = pd.concat([data_predict, first_data_predict])

In [107]:
all_data = pd.concat([total_crime_per_majorcat_per_month, data_predict])
all_data = all_data.sort_values(by = ['type', 'period'], ignore_index = True)
all_data

,period,major_category,total_crime,type
0,2008-01,Burglary,8396,original_data
1,2008-01,Criminal Damage,8061,original_data
2,2008-01,Drugs,5230,original_data
3,2008-01,Fraud or Forgery,2671,original_data
4,2008-01,Other Notifiable Offences,789,original_data
...,...,...,...,...
1183,2018-11,Other Notifiable Offences,1603,prediction
1184,2018-11,Robbery,1021,prediction
1185,2018-11,Sexual Offences,0,prediction
1186,2018-11,Theft and Handling,23097,prediction


In [114]:
# Import library plotly untuk visualisasi data
import numpy as np
import plotly.express as px

def plot_prediction(
    data : pd.DataFrame,
    value : str,
    judul : str,
    tipe : str,
    warna : str = 'black',
    showtrend : bool = False,
):
    fig = px.line(
        data_frame = data,
        x = data.index,
        y = value,
        color = tipe,
        color_discrete_map = {
            'original_data' : '#BA0001',
            'prediction' : '#FFAF00'
        } 
    )

    if(showtrend):
        # Hitung koefisien regresi linier
        coefficients = np.polyfit(range(len(data)), data[value], 1)
        slope = coefficients[0]
        intercept = coefficients[1]

        # Tambahkan garis regresi ke plot
        fig.add_scatter(
            x = data.index,
            y = range(len(data)) * slope + intercept,
            mode = 'lines',
            line = dict(
                color = '#F7B4BB',
                dash = 'dash'
            )
        )

    fig.update_layout(
        width = 950,
        height = 400,
        showlegend = False,
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        title = dict(
            text = judul,
            font = dict(
                size = 28,
                color = '#B0A7A7'
            ),
            y = 0.92,
            x = 0.5
        ),
        yaxis = dict(
            title = '',
            showgrid = False,
            showline = False,
            showticklabels = False,
            zeroline = False,
        ),
        margin = dict(
            t = 80,
            b = 10,
            r = 20
        )
    )

    fig.show()

In [115]:
for category in all_data['major_category'].unique():
    plot_prediction(
        data = all_data[all_data['major_category'] == category].set_index('period'),
        value = 'total_crime',
        tipe = 'type',
        judul = f"<b>Prediksi Total Kriminalitas<span style='color:#BA0001'> {category}</span></b><br><sup><sup>di London untuk 2 Tahun Kedepan (2017 - 2018)</sup></sup>",
        showtrend = True
    )

***Analisa***<br><br>
Dari hasil prediksi diperoleh informasi berikut :
* <span style='color:#F6F193'>Trend Positif</span> (Kenaikan Tindak Kriminal) : Terjadi pada jenis kejahatan <span style='color:#ed4d4d'><b><i>Other Notifiable Offences</b></i></span> (Tindak Kriminal Lain yang Harus Dilaporkan) dan <span style='color:#ed4d4d'><b><i>Violence Against the Person</b></i></span> (Kekerasan Terhadap Orang)

<ol>Kenaikan pada kategori ini mencakup tindak kekerasan fisik yang dilakukan terhadap individu atau kejahatan yang tidak spesifik tetapi penting untuk dicatat. Peningkatan dalam kategori ini menunjukkan bahwa kasus-kasus kekerasan seperti serangan, pemukulan, atau tindak kekerasan lainnya mengalami kenaikan. Ini adalah sinyal penting bahwa ada kebutuhan mendesak untuk mengatasi dan mengurangi insiden kekerasan di masyarakat.<br><br>
<span style='color:#FFAF00'><b>CTA</b></span> : Implementasikan strategi penegakan hukum yang lebih efektif untuk menangani dan mengurangi tindak kriminal yang meningkat, termasuk peningkatan patroli, intervensi, dan program pencegahan. Selain itu, lakukan program penyuluhan dan edukasi di komunitas untuk meningkatkan kesadaran tentang masalah kekerasan dan cara-cara melaporkan dan mencegah tindak kriminal.</ol>

* <span style='color:#F6F193'>Trend Negatif</span> (Penurunan Tindak Kriminal) : Terjadi pada jenis kejahatan <span style='color:#ed4d4d'><b><i>Burglary</b></i></span> (Pencurian dengan Paksaan), <span style='color:#ed4d4d'><b><i>Criminal Damage</b></i></span> (Kerusakan Kriminal), <span style='color:#ed4d4d'><b><i>Drugs</b></i></span> (Narkoba) dan <span style='color:#ed4d4d'><b><i>Robbery</b></i></span> (Perampokan)

<ol>Penurunan dalam kategori perampokan yaitu perampasan langsung barang dengan ancaman kekerasan, pencurian, narkoba dan perusakan properti telah menurun. Walaupun menurun tetap pastikan kontrol bahwa pencapaian positif ini dapat dipertahankan dan ditingkatkan lebih lanjut.<br><br>
<span style='color:#FFAF00'><b>CTA</b></span> : Selidiki faktor-faktor yang mungkin telah berkontribusi pada penurunan ini, seperti peningkatan keamanan, kebijakan pencegahan yang lebih baik, atau program rehabilitasi yang efektif. Kemudian, terus pantau untuk memastikan bahwa penurunan tersebut berkelanjutan dan tidak ada pergeseran pola ke jenis kejahatan lain.
</ol>

* <span style='color:#F6F193'>Tidak Menunjukan Trend</span> (Statisioner) : Terjadi pada jenis kejahatan <span style='color:#ed4d4d'><b><i>Fraud or Forgery</b></i></span> (Penipuan atau Pemalsuan), <span style='color:#ed4d4d'><b><i>Sexual Offences</b></i></span> (Tindak Kriminal Seksual) dan <span style='color:#ed4d4d'><b><i>Theft and Handling</b></i></span> (Pencurian dan Penanganan Barang Curian)

<ol>Sejak Maret 2008 hingga akhir 2016 tidak lagi ditemukan kasus penipuan / pemalsuan dan tindak kriminal seksual. Hal ini bisa saja terjadi dikarenakan mungkin ada peningkatan dalam penerapan kebijakan anti-penipuan dan pemalsuan, seperti perbaikan dalam sistem verifikasi dokumen, teknologi keamanan yang lebih baik, dan tindakan hukum yang lebih tegas ataupun kebijakan yang lebih ketat dan dukungan untuk korban kriminal seksual. Namun yang perlu dievaluasi adalah tindakan kriminal Pencurian dan Penanganan Barang Curian. Dari hasil visualisasi memang tidak tampak adanya trend kenaikan atau penurunan hal ini mungkin terjadi karena kebijakan atau tindakan preventif yang dilakukan belum optimal untuk mencegah kejahatan tersebut.<br><br>
<span style='color:#FFAF00'><b>CTA</b></span> : Evaluasi pola pencurian terutama di daerah rawan. Evaluasi pula kebijakan atau strategi yang telah diterapkan, dampaknya mungkin belum terlalu terlihat atau memang adanya kebijakan yang belum efektif untuk menekan angka pencurian.</ol>

---
<b>Bachtiyar M. A. © 2024</b>